# Nifty Daily data

In [ ]:
## Installing TA-Lib
!python -m pip install TA_Lib-0.4.24-cp310-cp310-win_amd64.whl

Processing c:\users\hp\intrade_ai\notebooks\ta_lib-0.4.24-cp310-cp310-win_amd64.whl
TA-Lib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.


You should consider upgrading via the 'c:\Users\HP\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
!python -m pip install nbformat

You should consider upgrading via the 'c:\Users\HP\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import talib
from talib import abstract

In [5]:
def get_missing_data_dates(stock_name,stock_data):
    
    holidays_file = "../Data/misc/holidays.csv"
    traded_on_holidays_file = "../Data/misc/traded_on_holidays.csv"
    traded_on_weekends_file = "../Data/misc/traded_on_weekends.csv"
    missing_data_file = "../Data/misc/missing_data.csv"
    
    stock_df = stock_data.copy()
    stock_df.drop(columns=['stock_code'],inplace=True)
    ## convert date to datetime format and make it as index
    stock_df['datetime'] =  pd.to_datetime(stock_df['datetime'], infer_datetime_format=True)
    stock_df = stock_df.set_index("datetime")

    ## Convert to  daily continous data
    stock_df_continous = stock_df.groupby(pd.Grouper(freq='1D')).agg({"open": "first", 
                                             "high": "max", 
                                             "low": "min", 
                                             "close": "last",
                                             "volume":"sum"})
    ## remove null data dates from continous data and capture those rows to na_data
    na_data = stock_df_continous[~stock_df_continous['open'].notna()]
    stock_df_continous = stock_df_continous[stock_df_continous['open'].notna()]
    stock_df_continous = stock_df_continous.sort_index(ascending=True)

    ## Data present on weekends
    traded_on_weekends = stock_df_continous[stock_df_continous.index.day_name().isin(['Saturday', 'Sunday'])]
    print(traded_on_weekends.shape)

    ## Convert holidays data to datetime and make it as index
    holidays = pd.read_csv(holidays_file)
    holidays['date'] = holidays['date'].astype('datetime64[ns]')
    holidays = holidays.set_index("date")

    ## Traded on holidays
    traded_on_holidays = stock_df_continous[stock_df_continous.index.isin(holidays.index)]

    ## remove weekends from null data of continous data
    na_data = na_data[~na_data.index.day_name().isin(['Saturday', 'Sunday'])]
    missing_data = na_data[~na_data.index.isin(holidays.index)]

    if os.path.exists(traded_on_holidays_file):
        a = pd.read_csv(traded_on_holidays_file)
    else:
        a = pd.DataFrame()
    if stock_name not in  a.columns:
        a[stock_name]= pd.Series([str(t) for t in traded_on_holidays.index.date])

    if os.path.exists(traded_on_weekends_file):
        b = pd.read_csv(traded_on_weekends_file)
    else:
        b = pd.DataFrame()
    if stock_name not in  b.columns:
        b[stock_name]= pd.Series([str(t) for t in traded_on_weekends.index.date])

    if os.path.exists(missing_data_file):
        c = pd.read_csv(missing_data_file)
    else:
        c = pd.DataFrame()
    if stock_name not in  c.columns:
        c[stock_name]= pd.Series([str(t) for t in missing_data.index.date])

    #a.to_csv(traded_on_holidays_file,index=False)
    #b.to_csv(traded_on_weekends_file,index=False)
    #c.to_csv(missing_data_file,index=False)

    return stock_df_continous


In [6]:
nifty_daily = pd.read_csv("../Data/Nifty/nifty_1day.csv")
nifty_daily_continous = get_missing_data_dates("Nifty",nifty_daily)
reliance_daily = pd.read_csv("../Data/Reliance/reliance_1day.csv")
reliance_daily_continous = get_missing_data_dates("Reliance",reliance_daily)

(14, 5)
(14, 5)


In [22]:
nifty_daily = pd.read_csv("../Data/Nifty/nifty_1day.csv")
nifty_daily.drop(columns=['stock_code'],inplace=True)
nifty_daily['datetime'] =  pd.to_datetime(nifty_daily['datetime'], infer_datetime_format=True)
cols = ['datetime','open','high','low','close']
# data = data[cols]
nifty_daily = nifty_daily.set_index("datetime")

In [23]:
nifty_daily_continous = nifty_daily.groupby(pd.Grouper(freq='1D')).agg({"open": "first", 
                                             "high": "max", 
                                             "low": "min", 
                                             "close": "last",
                                             "volume":"sum"})
na_data = nifty_daily_continous[~nifty_daily_continous['open'].notna()]
nifty_daily_continous = nifty_daily_continous[nifty_daily_continous['open'].notna()]
nifty_daily_continous = nifty_daily_continous.sort_index(ascending=True)

In [24]:
na_data

,open,high,low,close,volume
datetime,,,,,
2007-01-06,NaN,NaN,NaN,NaN,0
2007-01-07,NaN,NaN,NaN,NaN,0
2007-01-11,NaN,NaN,NaN,NaN,0
2007-01-13,NaN,NaN,NaN,NaN,0
2007-01-14,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...
2022-10-26,NaN,NaN,NaN,NaN,0
2022-10-29,NaN,NaN,NaN,NaN,0
2022-10-30,NaN,NaN,NaN,NaN,0


In [25]:
nifty_daily_continous

,open,high,low,close,volume
datetime,,,,,
2007-01-02,3966.25,4016.65,3964.95,4007.40,277443911
2007-01-03,4007.70,4029.80,3985.00,4024.05,336972857
2007-01-04,4027.30,4041.00,3979.80,3988.80,369793444
2007-01-05,3990.15,4011.45,3969.65,3983.40,363396365
2007-01-08,3983.30,3987.45,3913.00,3933.40,345887697
...,...,...,...,...,...
2022-11-01,18130.70,18175.80,18060.15,18145.40,248784199
2022-11-02,18177.90,18178.75,18048.65,18082.85,2053916079
2022-11-03,17968.35,18106.30,17959.20,18052.70,250997227


In [26]:
traded_on_weekends = nifty_daily_continous[nifty_daily_continous.index.day_name().isin(['Saturday', 'Sunday'])]
traded_on_weekends

,open,high,low,close,volume
datetime,,,,,
2009-10-17,5159.35,5176.80,5124.25,5141.80,210638083
2010-02-06,4712.75,4768.15,4712.75,4757.25,132000049
2012-01-07,4755.60,4759.40,4743.05,4746.90,90456550
2012-03-03,5360.05,5369.60,5353.40,5359.40,113199762
2012-04-28,5209.60,5216.40,5196.35,5209.00,77942285
2012-09-08,5343.65,5366.30,5343.45,5358.70,75696522
2013-05-11,6088.20,6114.55,6084.15,6107.25,75688341
2013-11-03,6332.05,6342.95,6311.15,6317.35,182578015
2014-03-22,6497.80,6502.65,6481.35,6494.90,90860320


In [27]:
## Holidays or weekends
holidays = pd.read_csv("../Data/holidays.csv")
trade_on_holiday = pd.read_csv("../Data/trade_on_holidays.csv")
trade_on_weekend = pd.read_csv("../Data/trade_on_weekends.csv")

In [28]:
holidays['date'] = holidays['date'].astype('datetime64[ns]')
holidays = holidays.set_index("date")

In [29]:
traded_on_holidays = nifty_daily_continous[nifty_daily_continous.index.isin(holidays.index)]
traded_on_holidays

,open,high,low,close,volume
datetime,,,,,
2007-11-09,5703.45,5794.20,5614.90,5663.25,177709487
2008-10-28,2526.20,2695.95,2526.20,2684.60,125137979
2009-03-27,3079.40,3123.35,3055.90,3108.65,879729349
2009-08-19,4457.75,4477.55,4353.45,4394.10,870511127
2010-11-05,6321.85,6338.50,6303.10,6312.45,239069253
2011-10-26,5214.95,5219.25,5196.15,5201.80,137221682
2012-10-26,5683.55,5697.20,5641.75,5664.30,625233411
2012-11-13,5689.70,5698.25,5660.35,5666.95,254963004
2013-11-14,6037.00,6101.65,6036.65,6056.15,527842208


In [30]:
holidays


,day,description
date,,
2007-01-01,Monday,Bakri ID
2007-01-26,Friday,Republic Day
2007-01-30,Tuesday,Moharram
2007-02-16,Friday,Mahashivratri
2007-03-27,Tuesday,Ram Navami
...,...,...
2022-08-15,Monday,Independence Day
2022-08-31,Wednesday,Ganesh Chaturthi
2022-10-05,Wednesday,Dussehra


In [31]:
na_data

,open,high,low,close,volume
datetime,,,,,
2007-01-06,NaN,NaN,NaN,NaN,0
2007-01-07,NaN,NaN,NaN,NaN,0
2007-01-11,NaN,NaN,NaN,NaN,0
2007-01-13,NaN,NaN,NaN,NaN,0
2007-01-14,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...
2022-10-26,NaN,NaN,NaN,NaN,0
2022-10-29,NaN,NaN,NaN,NaN,0
2022-10-30,NaN,NaN,NaN,NaN,0


In [32]:
na_data = na_data[~na_data.index.day_name().isin(['Saturday', 'Sunday'])]
na_data

,open,high,low,close,volume
datetime,,,,,
2007-01-11,NaN,NaN,NaN,NaN,0
2007-01-26,NaN,NaN,NaN,NaN,0
2007-01-30,NaN,NaN,NaN,NaN,0
2007-02-16,NaN,NaN,NaN,NaN,0
2007-02-26,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...
2022-08-09,NaN,NaN,NaN,NaN,0
2022-08-15,NaN,NaN,NaN,NaN,0
2022-08-31,NaN,NaN,NaN,NaN,0


In [33]:
missing_data = na_data[~na_data.index.isin(holidays.index)]
missing_data

,open,high,low,close,volume
datetime,,,,,
2007-01-11,NaN,NaN,NaN,NaN,0
2007-02-26,NaN,NaN,NaN,NaN,0
2007-03-16,NaN,NaN,NaN,NaN,0
2007-03-23,NaN,NaN,NaN,NaN,0
2007-04-27,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...
2017-07-21,NaN,NaN,NaN,NaN,0
2017-08-18,NaN,NaN,NaN,NaN,0
2017-08-24,NaN,NaN,NaN,NaN,0


In [34]:
l = missing_data.index.strftime('%Y-%m-%d')


In [35]:
[str(t) for t in missing_data.index.date]

['2007-01-11',
 '2007-02-26',
 '2007-03-16',
 '2007-03-23',
 '2007-04-27',
 '2007-05-31',
 '2007-07-05',
 '2007-07-09',
 '2007-08-10',
 '2007-11-15',
 '2007-12-06',
 '2007-12-19',
 '2008-02-06',
 '2008-02-26',
 '2008-04-04',
 '2008-05-05',
 '2008-05-09',
 '2008-05-12',
 '2008-05-29',
 '2008-05-30',
 '2008-06-02',
 '2008-06-06',
 '2008-06-09',
 '2008-06-27',
 '2008-07-25',
 '2008-08-22',
 '2008-09-16',
 '2008-09-17',
 '2008-10-03',
 '2008-10-29',
 '2008-11-27',
 '2008-12-31',
 '2009-01-12',
 '2009-02-06',
 '2009-02-19',
 '2009-04-29',
 '2009-04-30',
 '2009-06-26',
 '2009-06-30',
 '2009-07-13',
 '2009-09-17',
 '2009-10-13',
 '2010-02-03',
 '2010-04-15',
 '2010-05-21',
 '2010-06-29',
 '2010-07-28',
 '2010-08-27',
 '2010-10-04',
 '2010-10-05',
 '2011-01-20',
 '2011-02-02',
 '2011-07-08',
 '2011-10-28',
 '2012-02-10',
 '2012-02-24',
 '2012-10-04',
 '2013-11-15',
 '2014-01-09',
 '2014-06-11',
 '2014-12-04',
 '2015-04-16',
 '2015-04-21',
 '2015-07-15',
 '2016-06-16',
 '2016-06-17',
 '2016-07-

In [58]:
## function to get Pivot Points, Supports and Resistances
def PPSR(df):  
    PP = pd.Series((df['high'] + df['low'] + df['close']) / 3)  # main pivot level
    # pivot supports level(S1, S2, S3) and resistances level(R1, R2, R3)
    R1 = pd.Series(2 * PP - df['low'])  
    S1 = pd.Series(2 * PP - df['high'])  
    R2 = pd.Series(PP + df['high'] - df['low'])  
    S2 = pd.Series(PP - df['high'] + df['low'])  
    R3 = pd.Series(df['high'] + 2 * (PP - df['low']))  
    S3 = pd.Series(df['low'] - 2 * (df['high'] - PP))  
    psr = {'PP':PP, 'R1':R1, 'S1':S1, 'R2':R2, 'S2':S2, 'R3':R3, 'S3':S3}  
    PSR = pd.DataFrame(psr)
    return PSR

In [59]:
## function to get indicator values in dataframe
def indicator(data):
    stock_data = data.copy()
    stock_data['rsi'] = abstract.RSI(data, timeperiod=14) # Relative Strength Index
    stock_data['ema'] = abstract.EMA(data, timeperiod=10) # Exponential Moving Average
    stock_data['sma'] = abstract.SMA(data, timeperiod=20) # Simple Moving Average
    stock_data['UP_BB'], stock_data['MID_BB'], stock_data['LOW_BB'] = talib.BBANDS(stock_data['close'], \
        timeperiod=20, nbdevup=2, nbdevdn=2, matype=0) # Bollinger Bands of upper, middle and lower bands
    df = PPSR(stock_data) #calling PPSR function to get pivot points, supports and resistances
    stock_data = stock_data.join(df)
    return stock_data

In [61]:
nifty_indicator_data = indicator(nifty_daily_continous)
nifty_indicator_data

,open,high,low,close,volume,rsi,ema,sma,UP_BB,MID_BB,LOW_BB,PP,R1,S1,R2,S2,R3,S3
datetime,,,,,,,,,,,,,,,,,,
2007-01-02,3966.25,4016.65,3964.95,4007.40,277443911,NaN,NaN,NaN,NaN,NaN,NaN,3996.333333,4027.716667,3976.016667,4048.033333,3944.633333,4079.416667,3924.316667
2007-01-03,4007.70,4029.80,3985.00,4024.05,336972857,NaN,NaN,NaN,NaN,NaN,NaN,4012.950000,4040.900000,3996.100000,4057.750000,3968.150000,4085.700000,3951.300000
2007-01-04,4027.30,4041.00,3979.80,3988.80,369793444,NaN,NaN,NaN,NaN,NaN,NaN,4003.200000,4026.600000,3965.400000,4064.400000,3942.000000,4087.800000,3904.200000
2007-01-05,3990.15,4011.45,3969.65,3983.40,363396365,NaN,NaN,NaN,NaN,NaN,NaN,3988.166667,4006.683333,3964.883333,4029.966667,3946.366667,4048.483333,3923.083333
2007-01-08,3983.30,3987.45,3913.00,3933.40,345887697,NaN,NaN,NaN,NaN,NaN,NaN,3944.616667,3976.233333,3901.783333,4019.066667,3870.166667,4050.683333,3827.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-01,18130.70,18175.80,18060.15,18145.40,248784199,69.030689,17744.327157,17443.8000,18103.573882,17443.8000,16784.026118,18127.116667,18194.083333,18078.433333,18242.766667,18011.466667,18309.733333,17962.783333
2022-11-02,18177.90,18178.75,18048.65,18082.85,2053916079,66.453269,17805.876765,17503.5750,18167.471854,17503.5750,16839.678146,18103.416667,18158.183333,18028.083333,18233.516667,17973.316667,18288.283333,17897.983333
2022-11-03,17968.35,18106.30,17959.20,18052.70,250997227,65.189792,17850.753717,17542.4950,18238.551187,17542.4950,16846.438813,18039.400000,18119.600000,17972.500000,18186.500000,17892.300000,18266.700000,17825.400000


In [14]:
fig = px.line(x=nifty_indicator_data.index, y=nifty_indicator_data["ema"])
fig.show()

In [15]:
fig = px.line(x=nifty_indicator_data.index, y=nifty_indicator_data["rsi"])
fig.show()

In [11]:
fig = px.line(x=nifty_indicator_data.index, y=nifty_indicator_data["adx"])
fig.show()

In [17]:
reliance_indicator_data = indicator(reliance_daily_continous)
reliance_indicator_data

,open,high,low,close,volume,rsi,ema,adx
datetime,,,,,,,,
2007-01-02,318.47,321.00,317.37,319.85,1149902,NaN,NaN,NaN
2007-01-03,320.81,322.20,318.55,320.71,1489730,NaN,NaN,NaN
2007-01-04,321.98,324.33,318.94,319.76,3022423,NaN,NaN,NaN
2007-01-05,318.47,327.14,318.47,321.55,3047126,NaN,NaN,NaN
2007-01-08,322.92,322.92,316.50,318.90,2191355,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-11-01,2600.00,2600.00,2508.00,2529.85,6443999,58.563371,2475.730772,20.770523
2022-11-02,2532.30,2550.20,2515.00,2545.65,4145144,60.296427,2485.053335,21.322786
2022-11-03,2529.85,2559.55,2525.05,2555.15,3089026,61.343324,2494.399557,21.985554


In [18]:
fig = px.line(x=reliance_indicator_data.index, y=reliance_indicator_data["ema"])
fig.show()

In [21]:
fig = px.line(x=reliance_indicator_data.index, y=reliance_indicator_data["rsi"])
fig.show()

In [22]:
fig = px.line(x=reliance_indicator_data.index, y=reliance_indicator_data["adx"])
fig.show()